# Import modules

In [1]:
import numpy as np #works with fixing graphs env!
import pandas as pd
import trackpy
import os
import h5py
import time
import matplotlib as mpl
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from datetime import datetime

import mspt.particle_fitting as fit
import mspt.image_processing as img
import mspt.particle_detection as detect
import mspt.trajectory_analysis1 as traj
import mspt.plotting as plot
os.environ['NUMEXPR_MAX_THREADS'] = '32'

pytorch/CUDA not available or not correctly installed


# Table of contents
`A` [**Movies to process (.MP)**](#moviestoprocess) <br/>

`B` [**Movie processing**](#movieprocessing) <br/>
&emsp;   1 [Parameter settings](#parametersettings) <br/>
&emsp;   2 [Background removal, spot detection, and linking](#bgdetlink) <br/>
&emsp;   3 [Alternatively, load already processed movies](#preprocessedmovies) <br/>
&emsp;&emsp;   1 [Pre-processed movies to load](#loadpreprocessedmovies) <br/>
&emsp;&emsp;   2 [Parameter settings](#parametersettings2) <br/>
&emsp;&emsp;   3 [Spot detection and linking](#detlink) <br/>

`C` [**Trajectory analysis**](#traj) <br/>
&emsp;   1 [Parameter settings](#parametersettings3) <br/>
&emsp;   2 [Files to process (.CSV)](#filestoprocess_csv) <br/>
&emsp;   3 [Output file (.HDF5)](#outputfile) <br/>
&emsp;   4 [Fit MSD and JDD](#fitmsdjdd) <br/>
&emsp;   5 [Contrast-to-mass conversion](#contrasttomassconversion) <br/>
&emsp;   6 [Estimate particle density on membrane](#particledensity) <br/>

`D` [**Plotting**](#plotting) <br/>
&emsp;   1 [Input HDF5 file](#hdf5file) <br/>
&emsp;   2 [Load single data frame](#loaddata) <br/>
&emsp;   3 [Alternatively, pool multiple data frames](#pooldata) <br/>
&emsp;   4 [Generate correlation plot of mass and diffusion](#plotKDE) <br/>
&emsp;   5 [Save figure to PDF](#savePDF) <br/>

# `A` Movies to process (.mp) <a name="moviestoprocess"></a>

<div class="alert alert-block alert-success">

|Input|Output|
|:---|:---|
|directory (str)| MP files (list)| 

</div>

In [2]:
# Specify directory
#directory = os.path.normpath(r'/Users/jamesmclean/Documents/Uni/Laidlaw/Ralf_Research/code/moviestest') # set manually
directory = img.directoryDialog(os.getcwd()) # or via dialog

# Generate list of .mp or .h5 files to process
filepaths_data = img.find_filepaths(directory, extension='mp', exclude=None)
for file in filepaths_data:
    print(file)
    
assert len(filepaths_data) > 0, 'Cannot find any HDF5 files to process in current directory'

/Users/jamesmclean/Desktop/061_JM_2A_S4_0.01 nM SAv.mp
/Users/jamesmclean/Desktop/James_McLean/12072024/failed/032_JC_B_H20.mp
/Users/jamesmclean/Desktop/James_McLean/12072024/failed/029_JC_A_H20+HA(1000@0.1nM).mp
/Users/jamesmclean/Desktop/James_McLean/12072024/failed/034_JC_C_H20.mp
/Users/jamesmclean/Desktop/James_McLean/12072024/failed/028_JC_A_H20.mp
/Users/jamesmclean/Desktop/James_McLean/12072024/processed/019_JM_B_SLB+SAv+HA(1000@0.1nM)after refocus.mp
/Users/jamesmclean/Desktop/James_McLean/12072024/processed/025_JM_D_SLB+SAv+HA(1000@0.1nM).mp
/Users/jamesmclean/Desktop/James_McLean/12072024/processed/014_JM_B_SLB+SAv(0.04)add before.mp
/Users/jamesmclean/Desktop/James_McLean/12072024/processed/030_JC_A_H20+HA(1000@1nM).mp
/Users/jamesmclean/Desktop/James_McLean/12072024/processed/021_JM_C_SLB(control new location ) .mp
/Users/jamesmclean/Desktop/James_McLean/12072024/processed/010_JM_B_SLB+SAv(0.04nM).mp
/Users/jamesmclean/Desktop/James_McLean/12072024/processed/024_JM_D_SLB+

# `B` Movie processing <a name="movieprocessing"></a>

<div class="alert alert-block alert-info">
<h4><center> Parameters </center></h4>

|Load movie|Background removal|Spot detection|Trajectory linking|
|:---|:---|:---|:---|
|`batch_mode` (bool)  | `mode` (str)                  | `thresh` (float)     | `dmax` (float)| 
|`frame_range` (list) | `window_length` (int)         |`DoG_estimates` (dict)|`max_frames_to_vanish` (int)| 
|`navg` (int)         | `save_processed_movies` (bool)|                      | `minimum_trajectory_length` (int)| 
|                     | `parallel` (bool)             |                      | |
|                     |`GPU` (bool)                   |                      | |
</div>

<div class="alert alert-block alert-success">

|Input|Output|
|:---|:---|
|MP files (list)| background-processed movie (HDF5 file)| 
|               | detection results (CSV file)| 
|               | linked trajectories (CSV file)| 
</div>

## `B.1` Parameter settings <a name="parametersettings"></a>

In [3]:
# General parameters
batch_mode = True # Load mp file(s) on the fly without pop-up file dialog
frame_range = [] # Restrict analysis to certain frames, e.g. [0, 2000]. To analyze whole movie, leave list empty.
navg = 1 # Frame averaging, applied before background removal

# Background removal
mode = 'continuous_median' # Choose background removal strategy
window_length = 1001 # Set median window length
save_processed_movies = True # Save movies after background removal
parallel = True # Use multiple cores to perform background substraction. Applies only if GPU=False
GPU = False # Use GPU to perform background substraction (requires CUDA and pytorch). Applies only if parallel=False

# Spot detection
thresh = 0.00055 # Threshold paramter to mask candidate spots
DoG_estimates={ 'T' : 0.1423, 's' : 2.1436, 'sigma' : 1.2921 } # Initial guesses for PSF fitting

# Trajectory linking parameters
dmax = 4. # Maximum displacement of particles per frame (in pixels)
max_frames_to_vanish = 0 # Allow to link particles that where missed in these many frames
minimum_trajectory_length = 5 # Only keep particle that exist for at least this many frames

assert len(frame_range)==2 or len(frame_range)==0, 'frame_range is expected to be either of type [] or [int, int]'

## `B.2` Background removal, spot detection, and linking <a name="bgdetlink"></a>

In [4]:
for filename in filepaths_data:
    
    # Apply continuous median background removal
    frames, file = img.mp_reader(batch_mode=batch_mode,
                                 file_to_load=filename,
                                 frame_range=frame_range,
                                 mode=mode,
                                 navg=navg,
                                 window_length=window_length,
                                 parallel=parallel, 
                                 GPU=GPU)
    
    # Get name of video
    name = os.path.splitext(os.path.basename(file))[0]
    # Get current timestamp to prevent overwrite data
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # Save processed movies to HDF5 file
    if save_processed_movies:
        saving_folder_movie = os.path.splitext(file)[0] + '_{}{}'.format(mode[11:], window_length)
        processed_movie_file = os.path.join(saving_folder_movie,
                                            os.path.split(saving_folder_movie)[1] + '.h5')
        
        if not os.path.exists(saving_folder_movie):
            os.makedirs(saving_folder_movie)
        
        with h5py.File(processed_movie_file, 'w') as fn:
            fn.create_dataset('frames', data=frames)
            fn.create_dataset('window_length', data=window_length)
            if frame_range:
                fn.create_dataset('frame_range', data=frame_range)
            else:
                fn.create_dataset('frame_range', data=[0, frames.shape[0]])
        print('Saved processed movies to {}'.format(processed_movie_file))

        
    # Detect and fit candidate spots
    fitted_particles = fit.particle_fitter(frames,
                                           halfsize=window_length//2,
                                           thresh=thresh,
                                           frame_range=[],
                                           method='trust-ncg',
                                           DoG_estimates=DoG_estimates)     
    
    # Create folder to save processed data
    if save_processed_movies:
        detections_folder = os.path.join(saving_folder_movie,
                                         'thresh{}_fits{}'.format(thresh, timestamp))
    else:
        detections_folder = '{}_{}{}_thresh{}_fits{}'.format(os.path.splitext(file)[0], # Remove file extension
                                                             mode[11:], # Remove 'continuous_'
                                                             window_length,
                                                             thresh,
                                                             timestamp)   
        
    if not os.path.exists(detections_folder):
        os.makedirs(detections_folder)    
        



    # Save particle detections as csv file
    if len(frame_range) == 0:
        csv_name = '{}_all_frames'.format(name)
        detections_file = os.path.join(detections_folder,
                                       csv_name + '.csv')  
    else:
        csv_name = '{}_frames{}-{}'.format(name, frame_range[0], frame_range[1])
        detections_file = os.path.join(detections_folder,
                                       csv_name + '.csv')
    
    fitted_particles.to_csv(detections_file)
    print('Saved trajectory list to {}'.format(detections_file))

    
    # Link trajectories
    linked_trajectories = trackpy.link_df(fitted_particles, search_range=dmax, memory=max_frames_to_vanish)
    linked_trajectories = linked_trajectories.sort_values(by=['particle', 'frame'])
    trajectories_lenfilt = trackpy.filter_stubs(linked_trajectories, minimum_trajectory_length)

    trajectories_folder = os.path.join(detections_folder,
                                       'dmax{}_mem{}_fits{}'.format(dmax, max_frames_to_vanish, timestamp))
    
    if not os.path.exists(trajectories_folder):
        os.makedirs(trajectories_folder)

    trajectories_file = os.path.join(trajectories_folder,
                                     '{}_trajectories.csv'.format(csv_name))

#     # Workaround in Windows if filename is too long
#     if len(trajectories_file) >= 260: #max is 260
#         trajectories_file = '\\\\?\\'+ trajectories_file

    trajectories_lenfilt.to_csv(trajectories_file)
    print('Saved trajectory list to {}'.format(trajectories_file))

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=2000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



pytorch/CUDA not available or not correctly installed
Saved processed movies to /Users/jamesmclean/Desktop/James_McLean/12072024/failed/032_JC_B_H20_median1001/032_JC_B_H20_median1001.h5


Identifying particle candidates...:   0%|          | 0/10985 [00:00<?, ?frames/s]

402 particle candidates identified


Fitting particles...:   0%|          | 0/402 [00:00<?, ?candidate spots/s]

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 1 has 2 dimension(s)

## `B.3` Alternatively, load already processed movies <a name="preprocessedmovies"></a>

<div class="alert alert-block alert-info">
<h4><center> Parameters </center></h4>

|Spot detection|Trajectory linking|
|:---|:---|
| `thresh` (float)     | `dmax` (float)| 
|`DoG_estimates` (dict)|`max_frames_to_vanish` (int)| 
|                      | `minimum_trajectory_length` (int)| 
</div>

<div class="alert alert-block alert-success">

|Input|Output|
|:---|:---|
|HDF5 files (list)| detection results (CSV file)| 
|| linked trajectories (CSV file)| 

</div>

### `B.3.1` Pre-processed movies to load <a name="loadpreprocessedmovies"></a> 

In [ ]:
# Specify directory containing background-corrected movies
#directory = os.path.normpath(r'/Users/jamesmclean/Documents/Uni/Laidlaw/Ralf_Research/code/moviestest/004_2nM_HA500_in_8mM_MgCl2_dry_PDL_median1001') # set manually
directory = img.directoryDialog(os.getcwd()) # or via dialog

# Generate list of .h5 files to process
filepaths_data = img.find_filepaths(directory, extension='h5', exclude=None)
for file in filepaths_data:
    print(file)

assert len(filepaths_data) > 0, 'Cannot find any HDF5 files to process in current directory'

### `B.3.2` Parameter settings <a name="parametersettings2"></a>

In [ ]:
# Analyze subset of saved movie
frame_range = [] # Restrict analysis to certain frames, e.g. [0, 2000]. To analyze whole movie, leave list empty.

# Spot detection
thresh = 0.00055 # Threshold paramter to mask candidate spots
DoG_estimates={ 'T' : 0.1423, 's' : 2.1436, 'sigma' : 1.2921 } # Initial guesses for PSF fitting

# Trajectory linking parameters:
dmax = 4. # Maximum displacement of particles per frame (in pixels)
max_frames_to_vanish = 0 # Allow to link particles that where missed in these many frames
minimum_trajectory_length = 5 # Only keep particle that exist for at least this many frames

assert len(frame_range)==2 or len(frame_range)==0, 'frame_range is expected to be either of type [] or [int, int]'

### `B.3.3` Spot detection and linking <a name="detlink"></a> 

In [ ]:
for file in filepaths_data:
    # Load processed movies from HDF5 file
    with h5py.File(file, 'r') as h5_file:
        frames = np.asarray(h5_file['frames']).copy()
        window_length = np.asarray(h5_file['window_length']).copy()
        frame_range_h5 = np.asarray(h5_file['frame_range']).copy()
        if frame_range:
            assert frame_range[0] >= frame_range_h5[0], 'Requested frame range not contained in processed movie'
            assert frame_range[1] <= frame_range_h5[1], 'Requested frame range not contained in processed movie'
            frames = frames[ frame_range[0]-frame_range_h5[0] : frame_range[1]-frame_range_h5[0] ]
            
    print('Loaded processed movie {}'.format(file))
    
    # Get name of video
    name = os.path.splitext(os.path.basename(file))[0]
    # Get current timestamp to prevent overwrite data
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
        
    # Detect and fit candidate spots
    fitted_particles = fit.particle_fitter(frames,
                                           halfsize=window_length//2,
                                           thresh=thresh,
                                           method='trust-ncg',
                                           DoG_estimates=DoG_estimates)     
    
    # Create folder to save processed data
    saving_folder = os.path.dirname(file)
    detections_folder = os.path.join(saving_folder,
                                         'thresh{}_fits{}'.format(thresh, timestamp))
        
    if not os.path.exists(detections_folder):
        os.makedirs(detections_folder)    


    # Save particle detections as csv file
    if len(frame_range) == 0:
        csv_name = '{}_all_frames'.format(name)
        detections_file = os.path.join(detections_folder,
                                       csv_name + '.csv')  
    else:
        csv_name = '{}_frames{}-{}'.format(name, frame_range[0], frame_range[1])
        detections_file = os.path.join(detections_folder,
                                       csv_name + '.csv')
    
    fitted_particles.to_csv(detections_file)
    print('Saved trajectory list to {}'.format(detections_file))

    
    # Link trajectories
    linked_trajectories = trackpy.link_df(fitted_particles, search_range=dmax, memory=max_frames_to_vanish)
    linked_trajectories = linked_trajectories.sort_values(by=['particle', 'frame'])
    trajectories_lenfilt = trackpy.filter_stubs(linked_trajectories, minimum_trajectory_length)
    
    trajectories_folder = os.path.join(detections_folder,
                                       'dmax{}_mem{}_fits{}'.format(dmax, max_frames_to_vanish, timestamp))
    
    if not os.path.exists(trajectories_folder):
        os.makedirs(trajectories_folder)

    trajectories_file = os.path.join(trajectories_folder,
                                     '{}_trajectories.csv'.format(csv_name))

#     # Workaround in Windows if filename is too long
#     if len(trajectories_file) >= 260: #max is 260
#         trajectories_file = '\\\\?\\'+ trajectories_file

    trajectories_lenfilt.to_csv(trajectories_file)
    print('Saved trajectory list to {}'.format(trajectories_file))

# `C` Trajectory analysis <a name="traj"></a> 

<div class="alert alert-block alert-info">
<h4><center> Parameters </center></h4>

|Movie acquisition|
|:---|
| `frame_rate` (float)| 
| `pixel_size` (float)|
| `n_timelags_MSD` (int)|
| `n_timelags_JDD` (int)|
    
</div>

<div class="alert alert-block alert-success">

|Input|Output|
|:---|:---|
|CSV files (list)| analyzed trajectories (HDF5 file)| 

</div>

## `C.1` Parameter settings <a name="parametersettings3"></a> 

In [ ]:
# Movie acquisition parameters:
frame_rate = 199.8 # in Hz
pixel_size = 84.4 # in nm

# JDD analysis
n_timelags_MSD = None # Number of time lags to be used in MSD fitting. 
                      # If None, the optimal number of time lags is estimated.
n_timelags_JDD = None # Number of time lags to be used in JDD fitting. 
                      # If None, the same number of time lags is considered as in MSD fitting.

## `C.2` Files to process (.csv) <a name="filestoprocess_csv"></a> 

In [ ]:
# Specify directory with csv files
#directory = os.path.normpath(r'/Users/jamesmclean/Documents/Uni/Laidlaw/Ralf_Research/code/moviestest/004_2nM_HA500_in_8mM_MgCl2_dry_PDL_median1001/thresh0.00055_fits20240716_130640') # set manually
directory = img.directoryDialog(os.getcwd()) # or via dialog
assert os.path.isdir(directory), 'Directory "{}" does not exist'.format(directory)

# Create list of csv files containing trajectories
csv_files = traj.get_csv_files(directory)
for file in csv_files:
    print(file)

## `C.3` Output file (.hdf5) <a name="outputfile"></a> 

In [ ]:
# Specify filename of the results file 
filename = 'MSPT result.h5'
if os.path.isabs(filename):
    results_file = filename
else:
    results_file = os.path.join(os.path.normpath(directory),filename)

## `C.4` Fit mean squared displacement (MSD) and jump distance distribution (JDD) <a name="fitmsdjdd"></a> 

In [ ]:
# Fit trajectories
traj.fit_trajectories(csv_files,
                      os.path.join(results_file),
                      frame_rate=frame_rate,
                      pixel_size=pixel_size,
                      n_timelags_MSD=n_timelags_MSD,
                      n_timelags_JDD=n_timelags_JDD)

## `C.5` Contrast-to-mass conversion <a name="contrasttomassconversion"></a> 

In [ ]:
# Microscope-specific calibration parameters
slope = 32876.71 # in kDa/'contrast'
offset = -9.26 # in kDa

# Load HDF5 file where results of the trajectory analysis are stored
# Convert contrast to mass using the linear relationship obtained from calibration measurements
with pd.HDFStore(results_file, 'r+') as dfs:
    for idx, key in enumerate(dfs.keys()):
        print('Processing DataFrame {}/{}...'.format(idx+1,len(dfs.keys())))
        df = dfs[key].copy()
        #df['median_mass'] = slope * df['med_c'] + offset
        df['median_mass'] = traj.apply_calibration(df, slope=slope, offset=offset)
        dfs[key] = df.copy()
        
print('Added column "median_mass" to each DataFrame in the HDF5 container {}'.format(results_file))

## `C.6` Estimate particle density on membrane <a name="particledensity"></a> 

In [ ]:
# Load HDF5 file where results of the trajectory analysis are stored
# Calculate the number of particles present for each trajectory 'particle number (linked)'
with pd.HDFStore(results_file, 'r+') as dfs:
    for idx, key in enumerate(dfs.keys()):
        print('Processing DataFrame {}/{}...'.format(idx+1,len(dfs.keys())))
        df = dfs[key].copy()
        df['particle number (linked)'] = traj.calc_particle_number_linked(df)
        df['particle number (detected)'] = traj.calc_particle_number_detected(df, key)
        dfs[key] = df.copy()
        
print('Added column "particle number (linked)" and "particle number (detected)" to each DataFrame in the HDF5 container {}'.format(results_file))

# `D` Plotting <a name="plotting"></a> 

<div class="alert alert-block alert-info">
<h4><center> Plotting parameters </center></h4>

|X variable|Y variable| Axis limits | Filter trajectories |Misc
|:---|:---|:---|:---|:---|
|`median_mass`| `Deff_global` | `x_range` | `traj_length` | `figsize`  |
|`mean_mass`  | `Deff_JDD`    | `y_range` | `density`     | `n_levels` |
|`med_c`      | `D_MSD_n4`    |           |               | `cmap`     |
|`mean_c`     |               |           |               | `alpha`    |
|             |               |           |               | `show`     |
</div>

<div class="alert alert-block alert-success">

|Input|Output|
|:---|:---|
|MSPT results (DataFrame)| 2D-KDE plot (mpl figure/PDF)| 

</div>

## `D.1` Input HDF5 file <a name="hdf5file"></a> 

In [ ]:
# Specify HDF5 file containing MSPT results
#results_file = os.path.normpath(r"C:\Users\admin\Desktop\MSPT test\MSPT result.h5") # set manually
results_file = img.fileDialog(os.getcwd()) # or via file dialog
assert os.path.isfile(results_file), 'File "{}" does not exist'.format(results_file)

# Load data containers into memory as a dictionary
data = dict()
with pd.HDFStore(results_file, 'r') as dfs:
    for key in dfs.keys():
        data[key] = dfs[key].copy() 

## `D.2` Load single data frame <a name="loaddata"></a> 

In [ ]:
# Load single DataFrame
data_set_index = 0 # Specify data set index
key = list(data.keys())[data_set_index] # keys are trajectory CSV filenames
data_single = data[key].copy() # Load single DataFrame
print(data_single.shape)
print(data_single.columns)

## `D.3` Alternatively, pool multiple data frames <a name="pooldata"></a> 

In [ ]:
# Pool data to get a single DataFrame
data_pooled = pd.concat(data,axis=0)
print(data_pooled.shape)
print(data_pooled.columns)

## `D.4` Generate correlation plot of mass and diffusion <a name="plotKDE"></a> 

In [ ]:
# Plot data
fig, axs, df_kde, dat = plot.generate_2D_KDE(data_pooled,
                                             x='median_mass',
                                             y='Deff_MSD_JDD',
                                             x_range=(0,400), 
                                             y_range=(-1,1), # in log space
                                             figsize=(5,5),
                                             traj_length=5,
                                             density=None,
                                             n_levels=12,
                                             cmap=mpl.cm.gray_r,
                                             alpha=1.0,
                                             show=True)

## `D.5` Save figure to PDF <a name="savePDF"></a> 

In [ ]:
# Save plot to pdf
fig.savefig(r"C:\Users\admin\Desktop\JoVE Aldolase\MSPT_results_pooled.pdf", transparent=True, bbox_inches='tight')